In [1]:
import collections
import math

from IPython.display import display
import pandas as pd
import numpy as np

References:
- [Tensorflow Mechanics 101](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/tf/index.html)
- [Running Graphs](https://www.tensorflow.org/versions/r0.9/api_docs/python/client.html)
- [Building Graphs](https://www.tensorflow.org/versions/r0.9/api_docs/python/framework.html)


In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'0.9.0'

In [4]:
x = tf.constant(3.0)
y = tf.constant(4.0)
z = x + y

print 'z ==', z
print 'z.graph ==', z.graph
print 'x.graph == z.graph: ', z.graph == y.graph
print 'x.graph == default_graph(): ', x.graph == tf.get_default_graph()
sess = tf.Session()
# 7.0
print 'run(z) ==', sess.run(z)
# 7.0
print 'run(z) ==', sess.run(z)
sess.close()

# Error
# print 'run(z) ==', sess.run(z)

z == Tensor("add:0", shape=(), dtype=float32)
z.graph == <tensorflow.python.framework.ops.Graph object at 0x10d76af50>
x.graph == z.graph:  True
x.graph == default_graph():  True
run(z) == 7.0
run(z) == 7.0


In [5]:
# Call close() automatically by with-statement.
# Also, with-statement calls as_default() implicitly.
with tf.Session() as sess:
  print 'run(z) ==', sess.run(z)
  print 'z.eval() ==', z.eval()

run(z) == 7.0
z.eval() == 7.0


In [6]:
sess = tf.Session()

# This is error.
# z.eval()
print 'get_default_session', tf.get_default_session()

with sess.as_default():
    print 'get_default_session', tf.get_default_session(), tf.get_default_session() == sess
    print 'z.eval() ==', z.eval()

get_default_session None
get_default_session <tensorflow.python.client.session.Session object at 0x10d76a990> True
z.eval() == 7.0


In [7]:
x = tf.Variable(3.5)
y = x - tf.constant(4.0)
step = tf.train.GradientDescentOptimizer(0.1).minimize(tf.abs(y))
print step
print isinstance(step, tf.Operation)

print '===== default session ====='
with tf.Session():
    # FailedPreconditionError: Attempting to use uninitialized value 
    # x.eval()
    init = tf.initialize_all_variables()
    init.run()
    for i in xrange(10):
        step.run()
        print x.eval()

print '===== explicit session ====='
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)
for i in xrange(10):
    sess.run(step)
    print sess.run(x)  # sess does not have 'eval'. Use run.
sess.close()

print '===== explicit session as arg====='
sess = tf.Session()
init = tf.initialize_all_variables()
init.run(session=sess)
for i in xrange(10):
    step.run(session=sess)
    print x.eval(session=sess)
sess.close()

name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_Variable/ApplyGradientDescent"

True
===== default session =====
3.6
3.7
3.8
3.9
4.0
4.1
4.0
4.1
4.0
4.1
===== explicit session =====
3.6
3.7
3.8
3.9
4.0
4.1
4.0
4.1
4.0
4.1
===== explicit session as arg=====
3.6
3.7
3.8
3.9
4.0
4.1
4.0
4.1
4.0
4.1


# Graph
- A session is under a graph
- Methods to generate tensors (like constant) do not have 'graph' argument unlike eval/run and session.

In [8]:
g0 = tf.Graph()
g1 = tf.Graph()
with g0.as_default():
    c0 = tf.constant(3.0)
    c1 = tf.constant(4.0)
    sess0 = tf.Session()
with g1.as_default():
    c2 = tf.constant(4.0)
    sess1 = tf.Session()

print 'c0, c1, c2 =', c0, c1, c2
print 'Graph equality:', c0.graph == c1.graph, c0.graph == c2.graph

print 'c0.eval ==', c0.eval(session=sess0)

# Error!
# ValueError: Cannot use the given session to evaluate tensor: the tensor's graph is different from the session's graph.
# print 'c0.eval ==', c0.eval(session=sess1)

# 'c0 + c1' works even if g0 is not 'default'.
print '==== (c0 + c1) ===='
print (c0 + c1).eval(session=sess0)
print (c0 + c1).graph == c0.graph

# Error!
# ValueError: Tensor("Const:0", shape=(), dtype=float32) must be from the same graph as Tensor("Const:0", shape=(), dtype=float32).
# c2 = c0 + c1

sess0.close()
sess1.close()
print '=== finalize Graphs ==='
# Graphs are not finalized.
print g0.finalized, g1.finalized
g0.finalize()
g1.finalize()
print g0.finalized, g1.finalized

c0, c1, c2 = Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32) Tensor("Const:0", shape=(), dtype=float32)
Graph equality: True False
c0.eval == 3.0
==== (c0 + c1) ====
7.0
True
=== finalize Graphs ===
False False
True True


# What are Tensor, Variable and Operation?
- [tf.Tensor](https://www.tensorflow.org/versions/r0.9/api_docs/python/framework.html#Tensor)
- [tf.Variable](https://www.tensorflow.org/versions/r0.9/api_docs/python/framework.html#Variable)
- [tf.Operation](https://www.tensorflow.org/versions/r0.9/api_docs/python/framework.html#Operation)


In [9]:
tensor = pd.Series()
operation = pd.Series()
variable = pd.Series()

def check_type(label, v):
    tensor[label] = isinstance(v, tf.Tensor)
    operation[label] = isinstance(v, tf.Operation)
    variable[label] = isinstance(v, tf.Variable)

x = tf.constant(3.0)
y = tf.constant(4.0)
z = x + y
v = tf.Variable(1.0)
w = v + x
# This is Operation.
step = tf.train.GradientDescentOptimizer(0.1).minimize(w)

p = tf.placeholder(tf.float64)

mat = tf.constant([[1, 2], [3, 4]])
mul = tf.matmul(mat, mat)

check_type('x', x)
check_type('z', z)
check_type('p', p)
check_type('v', v)
check_type('w', w)
check_type('step', step)

check_type('mat', mat)
# It seems like the comment in
# https://www.tensorflow.org/versions/r0.9/api_docs/python/framework.html#Operation
# is wrong.
# "Objects of type Operation are created by calling a Python op constructor (such as tf.matmul())"
check_type('mul', mul)

df = pd.DataFrame(collections.OrderedDict([('Tensor', tensor), ('Variable', variable), ('Operation', operation)]))
df.applymap(lambda b:u'\u2713' if b else u'')

,Tensor,Variable,Operation
x,✓,,
z,✓,,
p,✓,,
v,,✓,
w,✓,,
step,,,✓
mat,✓,,
mul,✓,,


# Random and Session

In [10]:
mat0 = tf.random_normal([2,2])
mat1 = mat0 + 1
with tf.Session() as sess:
    # mat1.eval() != mat0.eval() + 1
    display(mat0.eval())
    display(mat1.eval())
    # mat1.eval() == mat0.eval() + 1
    display(sess.run((mat0, mat1)))

print '----- Random variable -------'
mat2 = tf.Variable(tf.random_normal([2, 2]))
with tf.Session():
    # Do not forget to initialize variables.
    tf.initialize_all_variables().run()
    # mat2.eval() == mat2.eval().
    display(mat2.eval())
    display(mat2.eval())

array([[ 0.00827565,  0.68900472],
       [-3.17785239, -0.5679031 ]], dtype=float32)

array([[ 1.86137152,  0.00334948],
       [ 1.00825703,  1.35924137]], dtype=float32)

[array([[-0.14897029, -0.71787143],
        [ 0.17828424,  1.41917324]], dtype=float32),
 array([[ 0.85102969,  0.28212857],
        [ 1.17828429,  2.41917324]], dtype=float32)]

----- Random variable -------


array([[ 1.51605272,  0.34388641],
       [ 0.87834281,  1.14176714]], dtype=float32)

array([[ 1.51605272,  0.34388641],
       [ 0.87834281,  1.14176714]], dtype=float32)